# Stage 2: What Should We Keep? - Feature Selection

This notebook follows from `eda.ipynb` (Stage 1) and implements a feature-selection pipeline that removes high missingness and near constant features, removes duplicate columns, uses mutual information (MI) to rank features (including categorical/boolean), and resolves high multicollinearity (|r| >= 0.85) by keeping the feature with higher MI with the target. The final output is <=> `max_features` features and a documented table with reasons and preprocessing notes.

In [12]:
# Parameters and imports
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import mutual_info_classif
import json

# Configurable parameters
data_path = 'data/US_Accidents_March23.csv'
missing_threshold = 0.40
corr_threshold = 0.85
max_features = 20
target = 'Severity'
random_state = 42
output_doc_csv = 'outputs/selected_features_doc.csv'
output_removed_json = 'outputs/removed_columns_stage2.json'

# Load dataset
df = pd.read_csv(data_path)
print('Loaded dataset shape:', df.shape)

Loaded dataset shape: (7728394, 46)


## 0) Before Feature Selection

In [13]:
print("Initial number of features:", df.shape[1])
print("Initial feature list:")

for col in df.columns:
    print("-", col)

Initial number of features: 46
Initial feature list:
- ID
- Source
- Severity
- Start_Time
- End_Time
- Start_Lat
- Start_Lng
- End_Lat
- End_Lng
- Distance(mi)
- Description
- Street
- City
- County
- State
- Zipcode
- Country
- Timezone
- Airport_Code
- Weather_Timestamp
- Temperature(F)
- Wind_Chill(F)
- Humidity(%)
- Pressure(in)
- Visibility(mi)
- Wind_Direction
- Wind_Speed(mph)
- Precipitation(in)
- Weather_Condition
- Amenity
- Bump
- Crossing
- Give_Way
- Junction
- No_Exit
- Railway
- Roundabout
- Station
- Stop
- Traffic_Calming
- Traffic_Signal
- Turning_Loop
- Sunrise_Sunset
- Civil_Twilight
- Nautical_Twilight
- Astronomical_Twilight


## 1) Remove high-missingness features (configurable)

In [14]:
missing_pct = df.isna().mean()
high_missing_cols = missing_pct[missing_pct > missing_threshold].sort_values(ascending=False)
print(f'Columns with > {missing_threshold*100:.0f}% missing: {len(high_missing_cols)}')
print(high_missing_cols.head(30))

removed = {'missingness': list(high_missing_cols.index)}
df_reduced = df.drop(columns=removed['missingness'])
print('Shape after removing high-missingness cols:', df_reduced.shape)


Columns with > 40% missing: 2
End_Lat    0.440294
End_Lng    0.440294
dtype: float64
Shape after removing high-missingness cols: (7728394, 44)
Shape after removing high-missingness cols: (7728394, 44)


## 2) Remove leakage & redundant features

In [15]:
leakage_and_redundant = [
    'End_Time',
    'Description',
    'Distance(mi)',
    'Wind_Chill(F)',
    'End_Lat', 'End_Lng',
    'Nautical_Twilight',
    'Astronomical_Twilight',
    'Civil_Twilight',
    'Airport_Code',
    'Street',
    'Zipcode'
]

leakage_and_redundant = [c for c in leakage_and_redundant if c in df_reduced.columns]

print("EDA-based columns removed:", leakage_and_redundant)

removed['eda_based'] = leakage_and_redundant
df_reduced = df_reduced.drop(columns=leakage_and_redundant)
print("Shape after EDA-based removal:", df_reduced.shape)

EDA-based columns removed: ['End_Time', 'Description', 'Distance(mi)', 'Wind_Chill(F)', 'Nautical_Twilight', 'Astronomical_Twilight', 'Civil_Twilight', 'Airport_Code', 'Street', 'Zipcode']
Shape after EDA-based removal: (7728394, 34)
Shape after EDA-based removal: (7728394, 34)


## 3) Remove near-constant / low-variance features

In [16]:
low_variance_cols = []
# flag columns with only 1 unique value
for col in df_reduced.columns:
    if df_reduced[col].nunique(dropna=False) <= 1:
        low_variance_cols.append(col)

# for numeric columns, also flag near-constant (e.g., >99% same value)
for col in df_reduced.select_dtypes(include=[np.number]).columns:
    top_frac = df_reduced[col].value_counts(normalize=True, dropna=False).iloc[0]
    if top_frac >= 0.99:
        if col not in low_variance_cols:
            low_variance_cols.append(col)

print('Near-constant or low-variance columns:', low_variance_cols)
removed['low_variance'] = low_variance_cols
df_reduced = df_reduced.drop(columns=low_variance_cols, errors='ignore')
print('Shape after low-variance removal:', df_reduced.shape)


Near-constant or low-variance columns: ['Country', 'Turning_Loop']
Shape after low-variance removal: (7728394, 32)
Shape after low-variance removal: (7728394, 32)


## 4) Prepare data for Mutual Information (impute & encode)

In [17]:
# Ensure target exists and do not drop it in reductions
if target not in df_reduced.columns:
    raise KeyError(f'Expected target column {target} not found in DataFrame')

# Separate features and target for MI calculation
X = df_reduced.drop(columns=[target])
y = df_reduced[target].copy()

# Identify column types
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
print('Numeric cols:', len(numeric_cols), 'Categorical/boolean cols:', len(categorical_cols))

# Impute numeric with median, categorical with most frequent
num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')
X_num = pd.DataFrame(num_imputer.fit_transform(X[numeric_cols]), columns=numeric_cols) if numeric_cols else pd.DataFrame()
X_cat = pd.DataFrame(cat_imputer.fit_transform(X[categorical_cols]), columns=categorical_cols) if categorical_cols else pd.DataFrame()

# Encode categoricals with OrdinalEncoder for MI (discrete encoding)
if not X_cat.empty:
    enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    X_cat_enc = pd.DataFrame(enc.fit_transform(X_cat), columns=categorical_cols)
else:
    X_cat_enc = pd.DataFrame()

# Combine for MI computation
X_for_mi = pd.concat([X_num, X_cat_enc], axis=1)
print('Shape for MI computation:', X_for_mi.shape)


Numeric cols: 8 Categorical/boolean cols: 23
Shape for MI computation: (7728394, 31)
Shape for MI computation: (7728394, 31)


## 5) Mutual Information (all features)

In [18]:
sample_size = 200_000
df_sample = df_reduced.sample(n=sample_size, random_state=random_state)

X_sample = df_sample.drop(columns=[target])
y_sample = df_sample[target]

# Identify columns
numeric_cols = X_sample.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X_sample.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

# Impute numeric
X_num_sample = pd.DataFrame(
    num_imputer.fit_transform(X_sample[numeric_cols]),
    columns=numeric_cols
) if numeric_cols else pd.DataFrame()

# Impute categorical
X_cat_sample = pd.DataFrame(
    cat_imputer.fit_transform(X_sample[categorical_cols]),
    columns=categorical_cols
) if categorical_cols else pd.DataFrame()

# Ordinal encode categoricals
if len(categorical_cols) > 0:
    X_cat_enc_sample = pd.DataFrame(
        enc.fit_transform(X_cat_sample),
        columns=categorical_cols
    )
else:
    X_cat_enc_sample = pd.DataFrame()

# Combine numeric + categorical for MI
X_for_mi = pd.concat([X_num_sample, X_cat_enc_sample], axis=1)

print("Shape used for MI:", X_for_mi.shape)

# Mutual Information computation ---
mi_scores = mutual_info_classif(
    X_for_mi,
    y_sample,
    discrete_features='auto',
    random_state=random_state
)

mi_series = pd.Series(mi_scores, index=X_for_mi.columns).sort_values(ascending=False)

print("Top MI features (all types):")
display(mi_series.head(40))

Shape used for MI: (200000, 31)
Top MI features (all types):
Top MI features (all types):


Start_Lng            0.123250
ID                   0.122493
Start_Lat            0.107457
Source               0.096653
Start_Time           0.092533
Weather_Timestamp    0.091158
City                 0.075466
County               0.063928
State                0.031241
Wind_Speed(mph)      0.028731
Weather_Condition    0.026030
Visibility(mi)       0.018395
Temperature(F)       0.018024
Wind_Direction       0.017849
Timezone             0.013382
Crossing             0.010896
Traffic_Signal       0.010044
Sunrise_Sunset       0.009419
Pressure(in)         0.007610
Stop                 0.002942
Junction             0.002224
Humidity(%)          0.002200
Amenity              0.001489
No_Exit              0.001449
Roundabout           0.001262
Station              0.000806
Precipitation(in)    0.000472
Railway              0.000309
Traffic_Calming      0.000106
Give_Way             0.000000
Bump                 0.000000
dtype: float64

## 6) Resolve high collinearity pairs (|r| >= corr_threshold) by keeping higher-MI feature

In [19]:
# Work with numeric columns' correlation
numeric_df = X_num.copy()
corr_matrix = numeric_df.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
pairs_to_check = []
for i in range(upper_tri.shape[0]):
    for j in range(i+1, upper_tri.shape[1]):
        val = upper_tri.iat[i, j]
        if pd.notna(val) and val >= corr_threshold:
            a = upper_tri.index[i]; b = upper_tri.columns[j]
            pairs_to_check.append((a, b, val))

print(f'Found {len(pairs_to_check)} highly correlated numeric pairs (|r|>={corr_threshold}):')
if len(pairs_to_check) > 0:
    print(pairs_to_check[:20])

# Decide which to drop: compare MI scores (if available), else variance
to_drop = set()
for a, b, r in pairs_to_check:
    mi_a = mi_series.get(a, 0.0)
    mi_b = mi_series.get(b, 0.0)
    # drop the one with lower MI (less relevant to target)
    if mi_a >= mi_b:
        to_drop.add(b)
    else:
        to_drop.add(a)

print('Numeric cols selected for drop due to collinearity:', list(to_drop)[:40])
removed['collinearity'] = list(to_drop)
# Drop from df_reduced (if present)
df_reduced = df_reduced.drop(columns=list(to_drop), errors='ignore')
print('Shape after collinearity-based drops:', df_reduced.shape)


Found 0 highly correlated numeric pairs (|r|>=0.85):
Numeric cols selected for drop due to collinearity: []
Shape after collinearity-based drops: (7728394, 32)
Shape after collinearity-based drops: (7728394, 32)


## 7) Evaluate road features (crosstabs) and include booleans in MI ranking

In [20]:
road_features = [c for c in ['Junction','Crossing','Traffic_Signal','Stop','Railway','Station','Amenity','Bump'] if c in df_reduced.columns]
print('Road features in dataset:', road_features)
for rf in road_features:
    print("\nCrosstab for:", rf)
    display(pd.crosstab(df_reduced[rf], df_reduced[target], normalize='index') * 100)


Road features in dataset: ['Junction', 'Crossing', 'Traffic_Signal', 'Stop', 'Railway', 'Station', 'Amenity', 'Bump']

Crosstab for: Junction


Severity,1,2,3,4
Junction,,,,
False,0.897562,80.226356,16.303375,2.572707
True,0.547308,72.660333,23.190313,3.602046



Crosstab for: Crossing


Severity,1,2,3,4
Crossing,,,,
False,0.710104,78.245423,18.254885,2.789589
True,2.139138,90.819364,5.497143,1.544355



Crosstab for: Traffic_Signal


...

Severity,1,2,3,4
Station,,,,
False,0.859072,79.358926,17.09892,2.683082
True,1.341251,91.152099,6.13568,1.370969



Crosstab for: Amenity


Severity,1,2,3,4
Amenity,,,,
False,0.864603,79.507879,16.971918,2.655600
True,1.431478,92.274794,4.183362,2.110366



Crosstab for: Bump


Severity,1,2,3,4
Bump,,,,
False,0.871625,79.662558,16.816196,2.649620
True,0.967558,89.470689,8.708025,0.853728


## 8) Final feature selection: top MI features up to `max_features`

In [21]:
sample_size = 200_000
df_sample = df_reduced.sample(n=sample_size, random_state=random_state)

X_sample = df_sample.drop(columns=[target])
y_sample = df_sample[target]

numeric_cols = X_sample.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X_sample.select_dtypes(include=['object','category','bool']).columns.tolist()

# Impute sampled data
X_num_sample = pd.DataFrame(num_imputer.transform(X_sample[numeric_cols]), columns=numeric_cols) if numeric_cols else pd.DataFrame()
X_cat_sample = pd.DataFrame(cat_imputer.transform(X_sample[categorical_cols]), columns=categorical_cols) if categorical_cols else pd.DataFrame()

# Encode categorical
if not X_cat_sample.empty:
    X_cat_enc_sample = pd.DataFrame(enc.transform(X_cat_sample), columns=categorical_cols)
else:
    X_cat_enc_sample = pd.DataFrame()

X_for_mi_final = pd.concat([X_num_sample, X_cat_enc_sample], axis=1)

# Compute MI on the sample
mi_scores = mutual_info_classif(X_for_mi_final, y_sample, discrete_features='auto', random_state=random_state)
mi_series = pd.Series(mi_scores, index=X_for_mi_final.columns).sort_values(ascending=False)

# Pick top features (keeping target)
top_features = mi_series.head(max_features - 1).index.tolist()
top_features.append(target)

df_final = df_reduced[top_features].copy()

print('Selected final feature count (including target):', len(df_final.columns))
print("\nSelected features:")
for i, feature in enumerate(df_final.columns.tolist(), start=1):
    print(f"{i}. {feature}")


Selected final feature count (including target): 20

Selected features:
1. Start_Lng
2. ID
3. Start_Lat
4. Source
5. Start_Time
6. Weather_Timestamp
7. City
8. County
9. State
10. Wind_Speed(mph)
11. Weather_Condition
12. Visibility(mi)
13. Temperature(F)
14. Wind_Direction
15. Timezone
16. Crossing
17. Traffic_Signal
18. Sunrise_Sunset
19. Pressure(in)
20. Severity


## 9) Documentation table and outputs

In [22]:
# Build documentation table with explicit reasons
all_cols = list(df.columns)
kept = set(df_final.columns.tolist())
rows = []
# Use raw dtypes from df_reduced
raw_dtypes = df_reduced.dtypes.to_dict()

for col in all_cols:
    if col == target:
        rows.append({'Feature': col, 'Kept?': 'Yes', 'Reason': 'Target', 'Preprocessing Needed': '-'})
        continue
    if col in kept:
        reason = 'Kept: high MI or not removed in previous steps'
        if raw_dtypes.get(col) in [np.float64, np.int64]:
            prep = "Scale"
        elif raw_dtypes.get(col) == 'bool':
            prep = "One-hot encode"
        elif raw_dtypes.get(col) == 'object':
            prep = "One-hot encode"
        else:
            prep = "-"
    else:
        # Determine why removed
        why = []
        for k, v in removed.items():
            if col in v:
                why.append(k)
        reason = 'Removed: ' + (', '.join(why) if why else 'Low MI')
        prep = '-'
    rows.append({'Feature': col, 'Kept?': 'Yes' if col in kept else 'No', 'Reason': reason, 'Preprocessing Needed': prep})

doc_table = pd.DataFrame(rows)
doc_table['Before_Count'] = len(all_cols)
doc_table['After_Count'] = len(df_final.columns)

# Save documentation
doc_table.to_csv(output_doc_csv, index=False)
with open(output_removed_json, 'w') as f: json.dump(removed, f, indent=2)

print('Documentation table saved to', output_doc_csv)
print('Removed columns saved to', output_removed_json)
display(doc_table.head(40))


Documentation table saved to outputs/selected_features_doc.csv
Removed columns saved to outputs/removed_columns_stage2.json


,Feature,Kept?,Reason,Preprocessing Needed,Before_Count,After_Count
0,ID,Yes,Kept: high MI or not removed in previous steps,One-hot encode,46,20
1,Source,Yes,Kept: high MI or not removed in previous steps,One-hot encode,46,20
2,Severity,Yes,Target,-,46,20
3,Start_Time,Yes,Kept: high MI or not removed in previous steps,One-hot encode,46,20
4,End_Time,No,Removed: eda_based,-,46,20
5,Start_Lat,Yes,Kept: high MI or not removed in previous steps,Scale,46,20
6,Start_Lng,Yes,Kept: high MI or not removed in previous steps,Scale,46,20
7,End_Lat,No,Removed: missingness,-,46,20
8,End_Lng,No,Removed: missingness,-,46,20
9,Distance(mi),No,Removed: eda_based,-,46,20
